In [1]:

# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import requests

# Import the API key.
from config import weather_api_key


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

731

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [7]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather['weather'][0]['description']
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,                          
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | grand gaube
Processing Record 2 of Set 1 | vana-voidu
City not found. Skipping...
Processing Record 3 of Set 1 | novikovo
Processing Record 4 of Set 1 | hermanus
Processing Record 5 of Set 1 | mufumbwe
Processing Record 6 of Set 1 | hobart
Processing Record 7 of Set 1 | usinsk
Processing Record 8 of Set 1 | provideniya
Processing Record 9 of Set 1 | avarua
Processing Record 10 of Set 1 | tiksi
Processing Record 11 of Set 1 | fairbanks
Processing Record 12 of Set 1 | bluff
Processing Record 13 of Set 1 | cabadiangan
Processing Record 14 of Set 1 | yerbogachen
Processing Record 15 of Set 1 | busselton
Processing Record 16 of Set 1 | los llanos de aridane
Processing Record 17 of Set 1 | east london
Processing Record 18 of Set 1 | jasper
Processing Record 19 of Set 1 | bintulu
Processing Record 20 of Set 1 | turukhansk
Processing Record 21 of Set 1 | vila do maio
Processing Record 22 of Set 1 | ushua

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Grand Gaube,MU,-20.0064,57.6608,75.60,65,4,10.00,clear sky
1,Novikovo,RU,51.1537,37.8879,83.91,46,1,1.10,clear sky
2,Hermanus,ZA,-34.4187,19.2345,60.80,61,48,10.71,scattered clouds
3,Mufumbwe,ZM,-13.6833,24.8000,79.93,21,19,6.62,few clouds
4,Hobart,AU,-42.8794,147.3294,53.26,76,75,6.91,moderate rain
5,Usinsk,RU,65.9939,57.5281,74.39,34,83,9.75,broken clouds
6,Provideniya,RU,64.3833,-173.3000,46.13,72,93,3.89,overcast clouds
7,Avarua,CK,-21.2078,-159.7750,71.65,68,20,6.91,few clouds
8,Tiksi,RU,71.6872,128.8694,54.36,80,98,2.66,overcast clouds
9,Fairbanks,US,64.8378,-147.7164,48.31,81,0,3.44,clear sky


In [10]:
# Create the output file (CSV).
output_data_file = "Weather_Database/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")